In [1]:
import os
import numpy as np
from utils.trainingUtils import TrainingUtils

2023-04-13 18:09:46.867581: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 18:09:48.261103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [2]:
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test2/pre_processing/npy"
original_path = os.path.join(working_dir, "original.npy")
npy_path = os.path.join(working_dir, "mask.npy")

In [3]:
print(os.listdir(working_dir))

['4-8-pred.npy', 'mask.npy', 'test.npy', 'original.npy']


In [4]:
images = TrainingUtils.load_file(original_path)
labels = TrainingUtils.load_file(npy_path)

In [5]:
images.shape

(1812, 512, 512)

In [6]:
# images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
# labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)

images = TrainingUtils.resize_image(images)
labels = TrainingUtils.resize_image(labels)

In [7]:
images.shape

(1812, 512, 512, 1)

In [8]:
perm = TrainingUtils.random_permutation(len(images))

In [9]:
perm

array([1344,  353,  603, ..., 1237,  116,  999])

In [10]:
images = TrainingUtils.randomize_data(images, perm)
labels = TrainingUtils.randomize_data(labels, perm)

In [11]:
images = TrainingUtils.individual_normalize(images)
labels = labels.astype(np.float64)

In [12]:
len(images)

1812

In [13]:
# X_train, X_val, X_test = TrainingUtils.train_val_test_split(images, 1200, 1700)
# y_train, y_val, y_test = TrainingUtils.train_val_test_split(labels, 1200, 1700)

X_train, X_val, X_test = TrainingUtils.train_val_test_split(images, 0.5, 0.3, 0.2)
y_train, y_val, y_test = TrainingUtils.train_val_test_split(labels, 0.5, 0.3, 0.2)

In [14]:
model = TrainingUtils.init_model()

2023-04-13 18:10:32.450993: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-13 18:10:32.451043: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: chimera13
2023-04-13 18:10:32.451048: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: chimera13
2023-04-13 18:10:32.451134: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 450.172.1
2023-04-13 18:10:32.451148: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 450.172.1
2023-04-13 18:10:32.451152: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 450.172.1


In [15]:
model = TrainingUtils.compile_model(model)

In [ ]:
history = TrainingUtils.training(model, X_train, y_train, X_val, y_val, batch=60, epochs=40)

training and prediction for batch size:   60 epochs:   40
Epoch 1/40


In [ ]:
TrainingUtils.plot_predict(history, model, X_test, y_test)